***Примитивный чат-бот, который способен отвечать на небольшое количество вопросов и простых сообщений на английском языке.***

*Например: 'Hello', 'Hey', 'Hi', 'How old are you?', 'What is your name?', 'What can you do?' и т.д. *

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pickle
import csv
import random

In [ ]:
def talk_to_lionelya_pepsi_primary(test_set_sentence):
    csv_file_path = "DataSet/chat_bot_df.csv"
    tfidf_vectorizer_pikle_path = "DataSet/tfidf_vectorizer.pickle"
    tfidf_matrix_train_pikle_path ="DataSet/tfidf_matrix_train.pickle"

    try:
        with open(tfidf_vectorizer_pikle_path, 'rb') as f:
            tfidf_vectorizer = pickle.load(f)
        with open(tfidf_matrix_train_pikle_path, 'rb') as f:
            tfidf_matrix_train = pickle.load(f)
    except FileNotFoundError:
        sentences = []
        with open(csv_file_path, "r", encoding='utf-8') as sentences_file:
            reader = csv.reader(sentences_file, delimiter=',')
            for row in reader:
                sentences.append(row[0])

        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix_train = tfidf_vectorizer.fit_transform(sentences)

        with open(tfidf_vectorizer_pikle_path, 'wb') as f:
            pickle.dump(tfidf_vectorizer, f)
        with open(tfidf_matrix_train_pikle_path, 'wb') as f:
            pickle.dump(tfidf_matrix_train, f)

    test_set = (test_set_sentence, "")
    tfidf_matrix_test = tfidf_vectorizer.transform(test_set)
    cosine = cosine_similarity(tfidf_matrix_test, tfidf_matrix_train)
    cosine = np.delete(cosine, 0)

    max_similarity = cosine.max()
    response_index = 0
    if max_similarity > 0.7:
        new_max = max_similarity - 0.01
        idx = np.where(cosine > new_max)
        response_index = random.choice(idx[0])
    else:
        response_index = np.where(cosine == max_similarity)[0][0] + 2

    with open(csv_file_path, "r", encoding='utf-8') as sentences_file:
        reader = csv.reader(sentences_file, delimiter=',')
        for idx, row in enumerate(reader, 1):
            if idx == response_index:
                return row[1], response_index

while True:
    sentence = input("Talk to Lionelya Pepsi: ")
    response_primary, line_id_primary = talk_to_lionelya_pepsi_primary(sentence)
    print(response_primary)
